In [ ]:
import numpy as np
import pandas as pd 


from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier

SEED = 1111

#tf.random.set_seed(SEED)
np.random.seed(SEED)

import datatable

datatable_frame = datatable.fread('../input/jane-street-market-prediction/train.csv')
#df_raw = pd.read_csv('../input/jane-street-market-prediction/train.csv')
df_raw = datatable_frame.to_pandas()

df_raw = df_raw.query('date > 85').reset_index(drop = True) 
df_raw = df_raw[df_raw['weight'] != 0]


df_raw['action'] = ((df_raw['resp'].values) > 0).astype(int)

In [ ]:
df_raw['action'].sum()/len(df_raw)

In [ ]:
df_train, df_test = train_test_split(df_raw, test_size=0.2, shuffle=True, random_state=150)

In [ ]:
features = [c for c in df_train.columns if "feature" in c]

#f_mean = np.mean(train[features[1:]].values,axis=0)
f_median = df_train[features].median(axis=0)

neutral_values = f_median

df_train.fillna(neutral_values,inplace=True)

In [ ]:
resp_cols = ['resp', 'resp_1', 'resp_2', 'resp_3', 'resp_4']

X_train = df_train.loc[:, df_train.columns.str.contains('feature')]
#y_train = (train.loc[:, 'action'])

y_train = np.stack([(df_train[c] > 0).astype('int') for c in resp_cols]).T

y_action_train = df_train['action'].to_numpy()

In [ ]:
df_test.fillna(neutral_values,inplace=True)

X_test = df_test.loc[:, df_test.columns.str.contains('feature')]
#y_train = (train.loc[:, 'action'])

y_test = np.stack([(df_test[c] > 0).astype('int') for c in resp_cols]).T

y_action_test = df_test['action'].to_numpy()

Random Forest

In [ ]:
rfc = RandomForestClassifier(n_estimators=50, max_depth=3, random_state=543)
rfc.fit(X_train, y_train)
five_preds = rfc.predict_proba(X_test)
five_preds = [arr[:,1] for arr in five_preds]  # arr[0] is the probability for class 0, arr[1] is the probability for class 1
five_preds_array = np.array(five_preds).T

from sklearn import metrics

th = 0.5

f_get_action = np.median
preds = f_get_action(five_preds_array, axis=1)
actions_predicted = np.where(preds >= th, 1, 0).astype(int)

print(preds.shape)
print(actions_predicted.shape)

metrics.accuracy_score(y_action_test, actions_predicted)

In [ ]:
f= np.median

models = rfc
import janestreet
from tqdm import tqdm
env = janestreet.make_env()
for (test_df, pred_df) in tqdm(env.iter_test()):
    if test_df['weight'].item() > 0:
        x_tt = test_df.loc[:, features].values
        if np.isnan(x_tt.sum()):
            #x_tt[:, 1:] = np.nan_to_num(x_tt[:, 1:]) + np.isnan(x_tt[:, 1:]) * f_mean
            x_tt = np.nan_to_num(x_tt) + np.isnan(x_tt) * neutral_values.values
        pred = np.mean([model.predict_proba(x_tt) for model in models],axis=0)
        pred = f(pred)
        pred_df.action = np.where(pred >= th, 1, 0).astype(int)
    else:
        pred_df.action = 0
    env.predict(pred_df)